<a href="https://colab.research.google.com/github/sean-otoole/HODD/blob/main/geneformer_training_with_dop_neuron_pop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scanpy anndata umap-learn python-igraph louvain h5py leidenalg > /dev/null 2>&1

In [2]:
from google.colab import drive
import os
import scanpy as sc
import pandas as pd
from scipy.io import mmread
import scipy.sparse
import pickle

In [3]:
# drive.mount('/content/drive')
# os.chdir("/content/drive/MyDrive/HODD/")

drive.flush_and_unmount()  # Unmount Google Drive
drive.mount('/content/drive')  # Remount it
os.chdir("/content/drive/MyDrive/HODD/")


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [4]:
!git lfs install

Git LFS initialized.


In [5]:
!git clone https://huggingface.co/ctheodoris/Geneformer

fatal: destination path 'Geneformer' already exists and is not an empty directory.


In [6]:
%cd Geneformer

/content/drive/MyDrive/HODD/Geneformer


In [7]:
!pip install .

Processing /content/drive/MyDrive/HODD/Geneformer
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664

In [8]:
from geneformer import TranscriptomeTokenizer

# Prepare the data using Geneformer Preprocessor

data_directory = '/content/drive/MyDrive/HODD/Geneformer_parkinsons_files'
output_path = data_directory
ouput_prefix = 'dopamine_subset_tokenized'

tk = TranscriptomeTokenizer({"disease_state": "disease_state_numeric"}, nproc=4)
tk.tokenize_data(data_directory, output_path, ouput_prefix, file_format="h5ad")

Tokenizing /content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad


100%|██████████| 34/34 [20:13<00:00, 35.71s/it]
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:797: ImplicitModificationWarning: Trying to modify index of attribute `.obs` of view, initializing view as actual.
  getattr(self, attr).index = value
/usr/local/lib/python3.11/dist-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/content/drive/MyDrive/HODD/Geneformer/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  for i in adata.var["ensembl_id_collapsed"][coding_miRNA_loc]
/content/drive/MyDrive/HODD/Geneformer/geneformer/tokenizer.py:498: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, inte

/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad has no column attribute 'filter_pass'; tokenizing all cells.


TypeError: unsupported operand type(s) for /: 'SparseCSRMatrixView' and 'Categorical'

In [19]:
import scanpy as sc

# Read the file
adata = sc.read_h5ad('/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad')

In [40]:
# Quick check of the data
# Convert the sparse matrix to a dense format (be mindful of memory usage with large datasets)
counts_dense = adata.layers["counts"].toarray()

# Convert the dense matrix to a pandas DataFrame
counts_df = pd.DataFrame(counts_dense, columns=adata.var_names, index=adata.obs_names)

# Display the DataFrame
print(counts_df.head())  # Display the first few rows

                                               ISG15  AGRN  SCNN1D  AURKAIP1  \
pPDsHSrSNxi3482d200429PosB_CAATGACTCTCTGCCA-1      1     0       1         8   
pPDsHSrSNxi3482d200429PosB_CTCTGGTCAGCTACTA-1      0     1       1         2   
pPDsHSrSNxi3482d200429PosB_CGAGGCTAGTGTACCT-1      1     0       0         2   
pPDsHSrSNxi3482d200429PosB_ACTACGAGTATGCAAA-1      0     0       0         0   
pPDsHSrSNxi3482d200429PosB_GACGCTGAGCCGTAAG-1      0     0       0         0   

                                               MRPL20  ANKRD65  SSU72  CALML6  \
pPDsHSrSNxi3482d200429PosB_CAATGACTCTCTGCCA-1       6        0     10       0   
pPDsHSrSNxi3482d200429PosB_CTCTGGTCAGCTACTA-1       6        0     10       0   
pPDsHSrSNxi3482d200429PosB_CGAGGCTAGTGTACCT-1       7        0      5       0   
pPDsHSrSNxi3482d200429PosB_ACTACGAGTATGCAAA-1       2        0      0       0   
pPDsHSrSNxi3482d200429PosB_GACGCTGAGCCGTAAG-1       0        0      1       0   

                                

In [98]:
!pip install pybiomart

import scanpy as sc
import pybiomart

annot = sc.queries.biomart_annotations(
    "hsapiens",
    ["ensembl_gene_id", "hgnc_symbol"],
)

gene_dict = annot.set_index("hgnc_symbol")["ensembl_gene_id"].to_dict()

In [99]:
annot

,ensembl_gene_id,hgnc_symbol
0,ENSG00000210049,MT-TF
1,ENSG00000211459,MT-RNR1
2,ENSG00000210077,MT-TV
3,ENSG00000210082,MT-RNR2
4,ENSG00000209082,MT-TL1
...,...,...
86401,ENSG00000235358,SCMH1-DT
86402,ENSG00000228067,LINC01740
86403,ENSG00000293271,SLC44A3-AS1
86404,ENSG00000310526,WASH7P


In [94]:
adata.X = adata.layers['counts']


In [74]:
annot = sc.queries.biomart_annotations(
    "hsapiens",
    ["ensembl_gene_id", "start_position", "end_position", "chromosome_name"],
).set_index("ensembl_gene_id")

ParseError: mismatched tag: line 62, column 2 (<string>)

In [55]:
from pybiomart import Server

# Connect to the Ensembl BioMart
server = Server("http://www.ensembl.org/biomart")

dir(server)

['_MART_XML_MAP',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_http_prefix',
 '_fetch_marts',
 '_host',
 '_mart_from_xml',
 '_marts',
 '_path',
 '_port',
 '_remove_trailing_slash',
 '_use_cache',
 'get',
 'host',
 'list_marts',
 'marts',
 'path',
 'port',
 'url',
 'use_cache']

In [41]:
# Quick check of the data
# Convert the sparse matrix to a dense format (be mindful of memory usage with large datasets)
counts_dense = adata.X.toarray()

# Convert the dense matrix to a pandas DataFrame
counts_df = pd.DataFrame(counts_dense, columns=adata.var_names, index=adata.obs_names)

# Display the DataFrame
print(counts_df.head())  # Display the first few rows

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'

In [42]:
adata.X

array([[ 0.15833633, -0.1656145 ,  0.22845554, ..., -0.00762537,
        -0.00762537, -0.00762537],
       [-0.27561887,  1.17038824,  0.40404857, ..., -0.00762537,
        -0.00762537, -0.00762537],
       [ 0.34252752, -0.1656145 , -0.27793383, ..., -0.00762537,
        -0.00762537, -0.00762537],
       ...,
       [-0.27561887, -0.1656145 , -0.27793383, ..., -0.00762537,
        -0.00762537, -0.00762537],
       [-0.27561887, -0.1656145 , -0.27793383, ..., -0.00762537,
        -0.00762537, -0.00762537],
       [-0.27561887, -0.1656145 , -0.27793383, ..., -0.00762537,
        -0.00762537, -0.00762537]])

In [ ]:
from geneformer import TranscriptomeTokenizer

In [ ]:
from geneformer import Geneformer

# Load pretrained Geneformer model
model = Geneformer.from_pretrained("geneformer")

# Fine-tune on your dataset
model.train(tokenized_data, output_dir="fine_tuned_geneformer")

In [ ]:
perturbed_data = preprocessor.apply_perturbation(adata.X, genes_to_perturb=["GeneA", "GeneB"])
perturbed_predictions = model.predict(perturbed_data)


In [ ]:
import scanpy as sc

file_path = "/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad"
data = sc.read_h5ad(file_path)

# Check the columns in `data.var`
print(data.var.columns)

# Check if `ensembl_id` is actually present
if "ensembl_id" in data.var.columns:
    print("ensembl_id column is present.")
else:
    print("ensembl_id column is missing.")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/HODD/Geneformer_parkinsons_files/dopamine_subset_pre_tokenization.h5ad', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)